In [149]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [150]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/tsomi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tsomi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Βήμα 1: Συλλογή Δεδομένων
Τα δεδομένα που χρησιμοποιήθηκαν για την υλοποίηση της εργασίας είναι βρίσκονται στον εξής σύνδεσμο: https://www.kaggle.com/datasets/sameersmahajan/people-wikipedia-data

In [151]:
df = pd.read_csv('in/pure_people_wiki.csv')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

## Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):

In [152]:
def tokenize(text):
    return text.split()

def remove_punctuation(words):
    cleaned_words = []
    for word in words:
        if word not in string.punctuation:
            cleaned_words.append(word)
    return cleaned_words

def filter_stop_words(words):
    filtered_words = []
    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    return filtered_words

def lemmatize_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

def preprocess_text(text):
    tokens = tokenize(text)                      # Step 1: Tokenization
    tokens = remove_punctuation(tokens)          # Step 2: Remove Punctuation
    tokens = filter_stop_words(tokens)           # Step 3: Remove Stop Words
    lemmatized_tokens = lemmatize_words(tokens)  # Step 4: Lemmatization
    return lemmatized_tokens

# Apply preprocessing to each text entry
df['processed_text'] = df['text'].apply(preprocess_text)

# Display the processed DataFrame
print(df[['URI', 'name', 'processed_text']].head())

                                               URI                 name  \
0        http://dbpedia.org/resource/Digby_Morrell        Digby Morrell   
1       http://dbpedia.org/resource/Alfred_J._Lewy       Alfred J. Lewy   
2        http://dbpedia.org/resource/Harpdog_Brown        Harpdog Brown   
3  http://dbpedia.org/resource/Franz_Rottensteiner  Franz Rottensteiner   
4               http://dbpedia.org/resource/G-Enka               G-Enka   

                                      processed_text  
0  [digby, morrell, born, 10, october, 1979, form...  
1  [alfred, j, lewy, aka, sandy, lewy, graduated,...  
2  [harpdog, brown, singer, harmonica, player, ac...  
3  [franz, rottensteiner, born, waidmannsfeld, lo...  
4  [henry, krvits, born, 30, december, 1974, tall...  


## Βήμα 3: Ευρετήριο (Indexing)
### α. Δημιουργία inverted index

In [153]:
corpus = {}

# Do the same for the first 1000 entries
# We enumerate the URI column of the dataframe so we can index it.
# Then we take its list of tokens and we count the amount of times each token appears
for i, uri in enumerate(df['URI'][:1000]):
    corpus[uri] = {tok: df['processed_text'][i].count(tok) for tok in df['processed_text'][i]}

# revert it back to a dataframe
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df[df.columns]


,aaron,lacrate,american,music,producer,recording,artist,dj,fashion,designer,...,investigationzuenir,janitor,denture,officeboy,universityhis,tribuna,imprensa,lacerda,managerowner,camus
http://dbpedia.org/resource/Aaron_LaCrate,5,12,1,7,1,2,2,3,5,2,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Abdel_Fattah_el-Sisi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Abdul_Salam_Azimi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Abdulalim_A._Shabazz,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Abu_Mustafa_al-Sheibani,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://dbpedia.org/resource/Yolande_Thibeault,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Yoshi_Wada,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Yoshihisa_Naruse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Zahid_Al-Sheikh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### β. Αποθήκευση του Inverted Index σε .csv αρχείο

In [154]:
# save the dataframe to a csv file
df.to_csv('out/results.csv', index = True)

Μετατροπή του .csv αρχείου σε dictionary

In [155]:
df = pd.read_csv('out/results.csv') # read the csv file
df.set_index(df.columns[0], inplace=True) # set the first column as the index
# print(df)

data_dict = df.to_dict(orient='index') # Convert DataFrame to dictionary

# # Print the first 5 entries of the dictionary
# for uri, terms in list(data_dict.items())[:5]:
#     print(f"Document URI: {uri}, Terms: {terms}") 

## 4. Μηχανή Αναζήτησης
### α. Επεξεργασία Ερωτήματος (Query Processing)

In [156]:
def search_query_dict(data_dict, query):
    tokens = query.split()  # Split the query into separate tokens
    print("Tokenized query: ", tokens)

    initial_term = tokens[0].lower()  # Retrieve the first term

    # Ensure the initial term exists in the data_dict
    if initial_term not in next(iter(data_dict.values())):  # Check the terms in the first document
        print(f"Term '{initial_term}' not found in the documents.")
        return []

    # Get URIs for the initial term (those documents where the term has a frequency > 0)
    results = {uri for uri, terms in data_dict.items() if terms.get(initial_term, 0) > 0}
    combined_results = results


    # Process additional terms with Boolean operators (AND, OR, NOT)
    i = 1
    while i < len(tokens):
        operator = tokens[i].upper()
        term = tokens[i + 1].lower()

        # Ensure the term exists in the data
        if term not in next(iter(data_dict.values())):
            print(f"Term '{term}' not found in the documents.")
            return []

        # Get URIs for the current term (those documents where the term has a frequency > 0)
        term_uris = {uri for uri, terms in data_dict.items() if terms.get(term, 0) > 0}

        # Apply the Boolean operators
        if operator == "AND":
            combined_results &= term_uris  # Intersection for AND
        elif operator == "OR":
            combined_results |= term_uris  # Union for OR
        elif operator == "NOT":
            combined_results -= term_uris  # Difference for NOT

        i += 2  # Move to the next pair

    return combined_results


**Εκτέλεση Boolean Query και εμφάνιση πλήθους αποτελεσμάτων (χωρίς κατάταξη)**

In [157]:
# Test the search_query_dict function with a boolean query (without ranking)
query = "musician AND jazz"
matching_uris = search_query_dict(data_dict, query)
print("Number of matching URIs: ", len(matching_uris))

for uri in list(matching_uris):
    print(f"Matching URI: {uri}")

Tokenized query:  ['musician', 'AND', 'jazz']
Number of matching URIs:  18
Matching URI: http://dbpedia.org/resource/Asahito_Nanjo
Matching URI: http://dbpedia.org/resource/Bob_Havens
Matching URI: http://dbpedia.org/resource/Shamek_Farrah
Matching URI: http://dbpedia.org/resource/Jamie_Shields_(musician)
Matching URI: http://dbpedia.org/resource/Gregg_Karukas
Matching URI: http://dbpedia.org/resource/Gwyn_Jay_Allen
Matching URI: http://dbpedia.org/resource/John_Edmondson_(musician)
Matching URI: http://dbpedia.org/resource/Vanessa_Ament
Matching URI: http://dbpedia.org/resource/Attila_Pacsay
Matching URI: http://dbpedia.org/resource/Tony_Garnier_(musician)
Matching URI: http://dbpedia.org/resource/Chris_Karan
Matching URI: http://dbpedia.org/resource/Dudley_Riggs
Matching URI: http://dbpedia.org/resource/Martin_Iveson
Matching URI: http://dbpedia.org/resource/Oren_Bloedow
Matching URI: http://dbpedia.org/resource/Mikko_Innanen_(musician)
Matching URI: http://dbpedia.org/resource/Dylan

### β. Κατάταξη αποτελεσμάτων

In [158]:
# Dictionary to store the scores
def search_query_dict_ranked(data_dict, query):
    combined_results = search_query_dict(data_dict, query)
    tokens = query.split()
    scores = {uri: 0 for uri in combined_results}

    # Calculate scores for the documents in the combined results
    for uri in combined_results:
        scores[uri] = sum(data_dict[uri].get(token.lower(), 0) for token in tokens if token.isalnum())

    # Sort documents by score in descending order
    ranked_results = sorted(combined_results, key=lambda uri: scores[uri], reverse=True)

    print("\nSearch Results:")
    print("-" * 40)
    if not ranked_results:
        print("No matching documents found.")
    else:
        print("Rank   Document                 Score")
        print("-" * 40)
        for rank, uri in enumerate(ranked_results, start=1):
           print(rank,"   ", uri,"  ", scores[uri])
        print("-" * 40)

    # Return the ranked list of matching URIs
    return ranked_results

In [159]:
# Test the search_query_dict function with a boolean query (without ranking)
matching_uris = search_query_dict_ranked(data_dict, query)

Tokenized query:  ['musician', 'AND', 'jazz']

Search Results:
----------------------------------------
Rank   Document             Score     
----------------------------------------
1      http://dbpedia.org/resource/Mikko_Innanen_(musician) 9         
2      http://dbpedia.org/resource/Shamek_Farrah 8         
3      http://dbpedia.org/resource/Gregg_Karukas 6         
4      http://dbpedia.org/resource/Sam_Stephenson_(writer) 6         
5      http://dbpedia.org/resource/Bob_Havens 5         
6      http://dbpedia.org/resource/Gwyn_Jay_Allen 4         
7      http://dbpedia.org/resource/John_Edmondson_(musician) 4         
8      http://dbpedia.org/resource/Tony_Garnier_(musician) 4         
9      http://dbpedia.org/resource/Dudley_Riggs 4         
10     http://dbpedia.org/resource/Jamie_Shields_(musician) 3         
11     http://dbpedia.org/resource/Chris_Karan 3         
12     http://dbpedia.org/resource/Martin_Iveson 3         
13     http://dbpedia.org/resource/Dylan_Cramer

In [160]:
import math
from collections import defaultdict

# Compute TF-IDF weights
def compute_tf_idf(data_dict):
    doc_count = len(data_dict)
    term_doc_frequency = defaultdict(int)

    for terms in data_dict.values():
        for term in terms:
            if terms[term] > 0:
                term_doc_frequency[term] += 1

    tf_idf_dict = {}
    for uri, terms in data_dict.items():
        tf_idf_dict[uri] = {}
        for term, freq in terms.items():
            tf = freq
            idf = math.log(doc_count / (1 + term_doc_frequency[term]))
            tf_idf_dict[uri][term] = tf * idf

    return tf_idf_dict


# Perform Boolean retrieval
def boolean_retrieval(data_dict, query):
    tokens = query.split()
    combined_results = set()

    initial_term = tokens[0].lower()
    if initial_term not in next(iter(data_dict.values())):
        return []

    combined_results = {uri for uri, terms in data_dict.items() if terms.get(initial_term, 0) > 0}
    i = 1

    while i < len(tokens):
        operator = tokens[i].upper()
        term = tokens[i + 1].lower()

        if term not in next(iter(data_dict.values())):
            return []

        term_uris = {uri for uri, terms in data_dict.items() if terms.get(term, 0) > 0}

        if operator == "AND":
            combined_results &= term_uris
        elif operator == "OR":
            combined_results |= term_uris
        elif operator == "NOT":
            combined_results -= term_uris

        i += 2

    return list(combined_results)


# Perform Vector Space Model retrieval
def vsm_retrieval(tf_idf_dict, query):
    query_terms = query.split()
    query_vector = defaultdict(float)

    for term in query_terms:
        query_vector[term.lower()] += 1

    scores = defaultdict(float)
    for uri, terms in tf_idf_dict.items():
        doc_vector = terms
        dot_product = sum(query_vector[term] * doc_vector.get(term, 0) for term in query_vector)
        query_magnitude = math.sqrt(sum(val**2 for val in query_vector.values()))
        doc_magnitude = math.sqrt(sum(val**2 for val in doc_vector.values()))
        if query_magnitude * doc_magnitude != 0:
            scores[uri] = dot_product / (query_magnitude * doc_magnitude)

    sorted_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    print("\n VSM Results ")
    print("-" * 40)
    print("Rank   Document         Score")
    print("-" * 40)
    for rank, (doc, score) in enumerate(sorted_results, start=1):
         print(rank,"  ", doc,"   ", score)
    print("-" * 40)

    return {doc: score for doc, score in sorted_results if score > 0}


# Perform Probabilistic Retrieval
def probabilistic_retrieval(data_dict, query):
    tokens = query.split()
    scores = defaultdict(float)
    doc_count = len(data_dict)
    
    for term in tokens:
        term_lower = term.lower()
        term_doc_count = 0
        for terms in data_dict.values():
            if terms.get(term_lower, 0) > 0:
                term_doc_count += 1

        ab = doc_count - term_doc_count + 0.5
        ba = term_doc_count + 0.5
        if ba > 0:
            idf = math.log(ab / ba)
        else:
            idf = 0 

        for uri, terms in data_dict.items():
            if terms.get(term_lower, 0) > 0:
                scores[uri] += idf

    sorted_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Print the results in a formatted way
    print("\nProbabilistic Retrieval Results (Scores):")
    print("-" * 40)
    print("Rank   Document         Score")
    print("-" * 40)
    for rank, (doc, score) in enumerate(sorted_results, start=1):
        print(rank,"  ", doc,"   ", score)
    print("-" * 40)

    result_dict = {}
    for doc, score in sorted_results:
        if score > 0:
            result_dict[doc] = score

    return result_dict

    return {doc: score for doc, score in sorted_results if score > 0}




## 5. Αξιολόγηση Συστήματος

In [161]:
# Evaluate the selected retrieval method
def evaluate_results(ground_truth, retrieved):
    # If the given input is a dict, then that means that there are two values contained in it (uri, score).
    # With the use of the .keys() method we can extract the URIs from the dictionary and discard the scores.
    if isinstance(retrieved, dict):
        retrieved = set(retrieved.keys())  # Extract document IDs if retrieved is a dictionary
    else:
        retrieved = set(retrieved)  # Convert list or other iterable to a set

    print("Retrieved Docs: ", retrieved)
    relevant = set(ground_truth)

    # calculate true positives, false positives, and false negatives
    tp = len(retrieved & relevant)  # true positives  - documents that are retrieved and are relevant
    fp = len(retrieved - relevant)  # false positives - documents that are retrieved but are not relevant
    fn = len(relevant - retrieved)  # false negatives - documents that are not retrieved but actually are relevant

    # Calculate precision, recall, and F1-score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Print calculated metrics
    print("\nEvaluation Metrics:")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1_score:.3f}")

ground_truth = search_query_dict_ranked(data_dict, query)

# Show option to the user
print("\nChoose Retrieval Method:")
print("1. Boolean Retrieval")
print("2. Vector Space Model")
print("3. Probabilistic Retrieval")
choice = int(input("Enter your choice (1/2/3): "))


if choice == 1:
    results = boolean_retrieval(data_dict, query)
    print("\nBoolean Retrieval Results:", results)
    evaluate_results(ground_truth, results)
elif choice == 2:
    # Remove logical operators for VSM
    vsm_query_tokens = []
    for token in query.split():
        if token.upper() not in {"AND", "OR", "NOT"}:
            vsm_query_tokens.append(token)
    vsm_query = " ".join(vsm_query_tokens)

    tf_idf_dict = compute_tf_idf(data_dict)
    results = vsm_retrieval(tf_idf_dict, vsm_query)
    evaluate_results(ground_truth, results)
elif choice == 3:
    # Remove logical operators for Probabilistic Retrieval
    prob_query_tokens = []
    for token in query.split():
        if token.upper() not in {"AND", "OR", "NOT"}:
            prob_query_tokens.append(token)
    prob_query = " ".join(prob_query_tokens)

    results = probabilistic_retrieval(data_dict, prob_query)
    evaluate_results(ground_truth, results)
else:
    print("Invalid choice!")
    results = []



Tokenized query:  ['musician', 'AND', 'jazz']

Search Results:
----------------------------------------
Rank   Document             Score     
----------------------------------------
1      http://dbpedia.org/resource/Mikko_Innanen_(musician) 9         
2      http://dbpedia.org/resource/Shamek_Farrah 8         
3      http://dbpedia.org/resource/Gregg_Karukas 6         
4      http://dbpedia.org/resource/Sam_Stephenson_(writer) 6         
5      http://dbpedia.org/resource/Bob_Havens 5         
6      http://dbpedia.org/resource/Gwyn_Jay_Allen 4         
7      http://dbpedia.org/resource/John_Edmondson_(musician) 4         
8      http://dbpedia.org/resource/Tony_Garnier_(musician) 4         
9      http://dbpedia.org/resource/Dudley_Riggs 4         
10     http://dbpedia.org/resource/Jamie_Shields_(musician) 3         
11     http://dbpedia.org/resource/Chris_Karan 3         
12     http://dbpedia.org/resource/Martin_Iveson 3         
13     http://dbpedia.org/resource/Dylan_Cramer